<font size="+3.8">Scanpy single-cell pre-processing</font>  
<font size="+1.5"></font>  

Aim: Preprocess human brain single-cell data from Garcia 2022 Nature  
Publication: https://www.nature.com/articles/s41586-022-04521-7

Also see https://nsun.shinyapps.io/scbbb/ (only showing ex-vivo dataset)

In [ ]:
from datetime import date
date.today().strftime('%d/%m/%Y')

In [ ]:
import os
import random
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sns

In [ ]:
import anndata as ad
import scanpy as sc
import scipy as sci
sc.settings.verbosity = 3

In [ ]:
#import loompy

In [ ]:
import utils

In [ ]:
main_dir='/cluster2/sfrerich/P6/P06_vasc_scRNAseq' # HPC

In [ ]:
dataset_name = "Garcia2022"
organism = "Human"

# Load data

Downloaded from http://compbio.mit.edu/scBBB/ (27/02/2023)

In [ ]:
# Turn rds to tsv
# /opt/R/4.0.0/bin/R
# library(data.table)
# main <- '/cluster2/sfrerich/P6/P06_vasc_scRNAseq'
# f <- readRDS(paste0(main, 'Garcia2022/preprocessed/brain.BBB.human.counts.rds'))
# f <- readRDS(paste0(main, 'Garcia2022/brain.BBB.human.counts.GEOsubmission.rds'))
# f <- readRDS(paste0(main, 'Garcia2022/ROSMAP.VascularCells.counts.rds'))
# m <- readRDS(paste0(main, 'Garcia2022/brain.BBB.human.meta.GEOsubmission.rds')) # export directly
# fd <- as.data.table(as.matrix(f))
# rownames(fd) <- as.data.frame(f@Dimnames[1])[,1] # dense
# colnames(fd) <- as.data.frame(f@Dimnames[2])[,1]
# fwrite(fd, paste0(main, 'Garcia2022/preprocessed/brain.BBB.human.counts.tsv'), sep="\t", row.names=TRUE, col.names=TRUE)
# fwrite(fd, paste0(main, 'Garcia2022/brain.BBB.human.counts.GEOsubmission.tsv'), sep="\t", row.names=TRUE, col.names=TRUE)
# fwrite(fd, paste0(main, 'Garcia2022/ROSMAP.VascularCells.counts.tsv'), sep="\t", row.names=TRUE, col.names=TRUE)
# fwrite(m, paste0(main, 'Garcia2022/brain.BBB.human.meta.GEOsubmission.txt'), sep="\t", col.names=TRUE)

In [ ]:
# from base folder -> brain.BBB.human.counts.GEOsubmission.tsv and brain.BBB.human.meta.GEOsubmission.txt
# not annotated / not used

## ROSMAP dataset

Nonpathological post-mortem brains

In [ ]:
# from base folder: ROSMAP study / annotated, vascular cells only

In [ ]:
ad_ROSMAP = sc.read_csv(os.path.join(main_dir,'Garcia2022','ROSMAP.VascularCells.counts.tsv'), delimiter="\t").transpose()

In [ ]:
ad_ROSMAP.obs_names_make_unique()
ad_ROSMAP.var_names_make_unique()

In [ ]:
ad_ROSMAP

In [ ]:
ad_ROSMAP.obs.head()

In [ ]:
meta_ROSMAP = pd.read_csv(os.path.join(main_dir,'Garcia2022','ROSMAP.VascularCells.meta.txt'), sep='\t')
meta_ROSMAP.head(2)

In [ ]:
pd.value_counts(meta_ROSMAP.celltype)

In [ ]:
pd.value_counts(meta_ROSMAP.subtype)

In [ ]:
ad_ROSMAP.obs = ad_ROSMAP.obs.join(meta_ROSMAP)

Check if data is normalized:

In [ ]:
ad_ROSMAP.layers

In [ ]:
ad_ROSMAP.X

In [ ]:
# show expression of 100 random genes (across all spots)
random_genes=random.sample(range(0, ad_ROSMAP.X.shape[1]), 100)
adata_sub = ad_ROSMAP[:,random_genes]
exp=pd.DataFrame(np.matrix(adata_sub.X))
# plot
pl1=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.3) # genes with 0 expression are excluded
pl1.set(xlim=(-0.5, 7),ylim=(0,0.007))
sns.set_theme(style='white')
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(4,4)})
sns.set_theme(style='white')
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)))
pl.set(xlim=(0, 10));
plt.show()

Data is not normalized, therefore normalize and log-transform

In [ ]:
ad_ROSMAP.layers["counts"] = ad_ROSMAP.X.copy() # save unnormalized raw RNA counts - retrieve via ad.X = ad.layers["counts"]

In [ ]:
sc.pp.normalize_total(ad_ROSMAP, inplace=True) # Normalize each spot by total counts over all genes, so that every cell has the same total count after normalization.

In [ ]:
sc.pp.log1p(ad_ROSMAP) # X = log(X + 1)

In [ ]:
# show expression of 100 random genes (across all spots)
adata_sub = ad_ROSMAP[:,random_genes]
exp=pd.DataFrame(np.matrix(adata_sub.X))
# plot
pl=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.5) # genes with 0 expression are excluded
pl.set(xlim=(-0.25, 3.5),ylim=(0,0.005));
plt.show()

In [ ]:
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)));
pl.set(xlim=(0, 10));
plt.show()

In [ ]:
ad_ROSMAP.layers["normalized"] = ad_ROSMAP.X.copy() # retrieve via adata.X = adata.layers["normalized"]

In [ ]:
# Identify highly-variable genes
sc.pp.highly_variable_genes(ad_ROSMAP)
sc.pl.highly_variable_genes(ad_ROSMAP)

In [ ]:
ad_ROSMAP

In [ ]:
ad_ROSMAP.layers

In [ ]:
pd.value_counts(meta_ROSMAP.celltype)

In [ ]:
# Renaming
old_to_new = {'aEndo':'aECs',
              'capEndo':'capECs',
              'vEndo':'vECs',
              'Per1':'Pericytes',
              'Per2':'Pericytes',
              'Per3':'Pericytes',
              'Fib1':'Fibroblasts',
              'Fib2':'Fibroblasts',
              'Fib3':'Fibroblasts',
              'aSMC':'SMCs',
              'vSMC':'SMCs'}
ad_ROSMAP.obs['clusters2'] = (ad_ROSMAP.obs['subtype'].map(old_to_new).astype('category'))

In [ ]:
# Renaming
old_to_new = {'Endo':'ECs',
              'Per':'Pericytes',
              'Fib':'Fibroblasts',
              'SMC':'SMCs'}
ad_ROSMAP.obs['clusters'] = (ad_ROSMAP.obs['celltype'].map(old_to_new).astype('category'))

### Dim Reduction

Nonpathological post-mortem brains

In [ ]:
sc.tl.pca(ad_ROSMAP)
sc.pl.pca_variance_ratio(ad_ROSMAP)

In [ ]:
sc.pp.neighbors(ad_ROSMAP)
sc.tl.umap(ad_ROSMAP)

In [ ]:
sc.pl.umap(ad_ROSMAP, color=['clusters','clusters2'], wspace=0.5)

In [ ]:
ad_ROSMAP.obs.clusters2.value_counts()

### Marker genes

In [ ]:
sc.tl.rank_genes_groups(ad_ROSMAP, 'clusters', method='wilcoxon')

In [ ]:
sc.pl.rank_genes_groups_dotplot(ad_ROSMAP,n_genes=7)

### Vascular marker genes

In [ ]:
# Basic QC metrics
ad_ROSMAP.var['mt'] = ad_ROSMAP.var_names.str.startswith('MT-') 
sc.pp.calculate_qc_metrics(ad_ROSMAP, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(ad_ROSMAP, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],jitter=0.4, multi_panel=True)

In [ ]:
# DEA requires HPC
sc.tl.rank_genes_groups(ad_ROSMAP, 'clusters', method='wilcoxon')

In [ ]:
sc.pl.rank_genes_groups(ad_ROSMAP, n_genes=20, sharey=False)

Export DEG table

In [ ]:
# export DEG table
celltypes = ad_ROSMAP.obs.clusters.unique().tolist()
merged_df = pd.DataFrame()
for group in celltypes:
    rank_df = rank_genes_groups_df(adata=ad_ROSMAP, group=group, adj_pval_cutoff=None, log2fc_cutoff=None, sortby='zscore')
    merged_df = pd.concat([merged_df, rank_df])
    merged_df['reference'] = 'all other clusters'
merged_df.head(3)

In [ ]:
merged_df.group.unique().tolist()

In [ ]:
# export as csv
merged_df.to_csv(os.path.join(main_dir,'results',date.today().strftime("%Y%m%d")+'_DE_genes_Garcia2022.csv'),sep=';')

In [ ]:
sc.tl.rank_genes_groups(ad_ROSMAP, 'clusters2', method='wilcoxon')

In [ ]:
sc.pl.rank_genes_groups(ad_ROSMAP, n_genes=20, sharey=False)

Export DEG table

In [ ]:
# export DEG table
celltypes = ad_ROSMAP.obs.clusters2.unique().tolist()
merged_df = pd.DataFrame()
for group in celltypes:
    rank_df = rank_genes_groups_df(adata=ad_ROSMAP, group=group, adj_pval_cutoff=None, log2fc_cutoff=None, sortby='zscore')
    merged_df = pd.concat([merged_df, rank_df])
    merged_df['reference'] = 'all other clusters'
merged_df.head(3)

In [ ]:
merged_df.group.unique().tolist()

In [ ]:
# export as csv
merged_df.to_csv(os.path.join(main_dir,'P06_vasc_scRNAseq','Garcia2022','DEA',date.today().strftime("%Y%m%d")+'_DE_genes_Garcia2022_incl_zonation.csv'),sep=';')

### Focus on: Foxf2

In [ ]:
gene="FOXF2"

In [ ]:
sc.pl.dotplot(ad_ROSMAP, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(ad_ROSMAP, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

## Ex-vivo dataset

Healthy tissue from epilepsy patients

In [ ]:
# from preprocessed folder

In [ ]:
adata = sc.read_csv(os.path.join(main_dir,'Garcia2022','preprocessed','brain.BBB.human.counts.tsv'), delimiter="\t").transpose()

In [ ]:
adata.obs_names_make_unique()
adata.var_names_make_unique()

In [ ]:
adata

In [ ]:
adata.var.head()

In [ ]:
adata.obs.head()

In [ ]:
# Add metadata

In [ ]:
meta = pd.read_csv(os.path.join(main_dir,'Garcia2022','preprocessed','brain.BBB.human.meta.txt'), sep='\t')
meta.head(2)

In [ ]:
meta.shape

In [ ]:
meta2 = pd.read_csv(os.path.join(main_dir,'Garcia2022','preprocessed','brain.BBB.human.vascular.final.Jan2022.metadata.txt'), sep='\t')
meta2.index = meta2["orig.ident"]
meta2.index.name = None
meta2.head(2)

In [ ]:
meta2.shape

In [ ]:
pd.value_counts(meta2.cellsubtype)

In [ ]:
meta = meta.join(meta2[["cellsubtype"]])

In [ ]:
meta['clusters'] = meta['cellsubtype'].fillna(meta['celltype'])

In [ ]:
meta.head(2)

In [ ]:
adata.obs = adata.obs.join(meta)

In [ ]:
adata.obs.columns

In [ ]:
adata.obs.head(2)

In [ ]:
pd.value_counts(adata.obs["snRNAPreparation"])

In [ ]:
pd.value_counts(meta.SampleID)

In [ ]:
pd.value_counts(adata.obs.PatientID)

In [ ]:
pd.value_counts(adata.obs.celltype)

In [ ]:
pd.value_counts(adata.obs.cellsubtype)

In [ ]:
pd.value_counts(adata.obs.clusters)

In [ ]:
# Renaming
old_to_new = {'aEndo':'aECs',
              'Astro':'Astrocytes',
              'capEndo':'capECs',
              'vEndo':'vECs',
              'Per1':'Pericytes',
              'Per2':'Pericytes',
              'Microglia':'Microglia',
              'Ex':'Neurons',
              'In':'Neurons',
              'OPC':'OPCs',
              'Fib1':'Fibroblasts',
              'Fib2':'Fibroblasts',
              'Fib3':'Fibroblasts',
              'aSMC':'SMCs',
              'vSMC':'SMCs',
              'Oligo':'Oligos'}
adata.obs['clusters2'] = (adata.obs['clusters'].map(old_to_new).astype('category'))

In [ ]:
# Renaming
old_to_new = {'aEndo':'ECs',
              'Astro':'Astrocytes',
              'capEndo':'ECs',
              'vEndo':'ECs',
              'Per1':'Pericytes',
              'Per2':'Pericytes',
              'Microglia':'Microglia',
              'Ex':'Neurons',
              'In':'Neurons',
              'OPC':'OPCs',
              'Fib1':'Fibroblasts',
              'Fib2':'Fibroblasts',
              'Fib3':'Fibroblasts',
              'aSMC':'SMCs',
              'vSMC':'SMCs',
              'Oligo':'Oligos'}
adata.obs['clusters'] = (adata.obs['clusters'].map(old_to_new).astype('category'))

In [ ]:
del meta

In [ ]:
# check if data is normalized:

In [ ]:
adata.layers

In [ ]:
adata.X.max()

In [ ]:
# show expression of 100 random genes (across all spots)
random_genes=random.sample(range(0, adata.X.shape[1]), 100)
adata_sub = adata[:,random_genes]
exp=pd.DataFrame(np.matrix(adata_sub.X))
# plot
pl1=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.3) # genes with 0 expression are excluded
pl1.set(xlim=(-0.5, 7),ylim=(0,0.007))
sns.set_theme(style='white')
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(4,4)})
sns.set_theme(style='white')
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)))
pl.set(xlim=(0, 10));
plt.show()

Data is not normalized, therefore normalize and log-transform

In [ ]:
adata.layers["counts"] = adata.X.copy() # save unnormalized raw RNA counts - retrieve via ad.X = ad.layers["counts"]

In [ ]:
sc.pp.normalize_total(adata, inplace=True) # Normalize each spot by total counts over all genes, so that every cell has the same total count after normalization.

In [ ]:
sc.pp.log1p(adata) # X = log(X + 1)

In [ ]:
# show expression of 100 random genes (across all spots)
adata_sub = adata[:,random_genes]
exp=pd.DataFrame(np.matrix(adata_sub.X))
# plot
pl=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.5) # genes with 0 expression are excluded
pl.set(xlim=(-0.25, 3.5),ylim=(0,0.005));
plt.show()

In [ ]:
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)));
pl.set(xlim=(0, 10));
plt.show()

In [ ]:
adata.layers["normalized"] = adata.X.copy() # retrieve via adata.X = adata.layers["normalized"]

In [ ]:
# Identify highly-variable genes
sc.pp.highly_variable_genes(adata)
sc.pl.highly_variable_genes(adata)

In [ ]:
adata

In [ ]:
adata.layers

### Dim Reduction

In [ ]:
sc.tl.pca(adata)
sc.pl.pca_variance_ratio(adata)

In [ ]:
sc.pp.neighbors(adata)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color='clusters')

### Marker genes

In [ ]:
sc.tl.rank_genes_groups(adata, 'clusters')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata,n_genes=5)

### Focus on: Foxf2

In [ ]:
gene="FOXF2"

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

## Merge

In [ ]:
adata = ad.concat([adata, ad_ROSMAP], join="inner") # ignore batch effects because data is pre-annotated

In [ ]:
adata

# Focus on: Foxf2

In [ ]:
gene="FOXF2"

## Excl zonation

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
utils.summarize_gene_expression(adata = adata, gene = gene, groupby = "clusters", 
                                study_name = dataset_name, organism = organism,
                                export = True, output_dir = os.path.join(main_dir, "Foxf2_summarized")
                               )

## Incl zonation

In [ ]:
utils.summarize_gene_expression(adata = adata, gene = gene, groupby = "clusters2", 
                                study_name = dataset_name, organism = organism,
                                export = True, output_dir = os.path.join(main_dir, "Foxf2_summarized", "incl_zonation")
                               )

# Focus on: Other genes

In [ ]:
target_genes = ["Foxo1", "Tek", "Nos3", "Htra1", "Egfl8", "Flt1", "Kdr", "Ptprb", "Nrp1", "Nrp2", "Efnb2", "Itgb1", "Itga6", "Angpt2", "Cdh5", "Cldn5", "Ocln", "Ctnnb1"]

In [ ]:
target_genes = [gene.upper() for gene in target_genes]
target_genes

## Excl zonation

In [ ]:
groupby = "clusters"

In [ ]:
other_genes_results = {
    gene: utils.summarize_gene_expression(adata, gene, study_name = dataset_name, organism = organism, groupby = groupby, 
                                          output_dir=os.path.join(main_dir, "Other_genes_summarized"), export=True
                                         ) for gene in target_genes
}

In [ ]:
# some plots

In [ ]:
sc.pl.matrixplot(adata, [target_genes[0]], groupby=groupby, swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [target_genes[0]], groupby=groupby, swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [target_genes[1]], groupby=groupby, swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [target_genes[1]], groupby=groupby, swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

## Incl zonation

In [ ]:
groupby = "clusters2"

In [ ]:
other_genes_results = {
    gene: utils.summarize_gene_expression(adata, gene, study_name = dataset_name, organism = organism, groupby = groupby, 
                                          output_dir=os.path.join(main_dir, "Other_genes_summarized", "incl_zonation"), export=True
                                         ) for gene in target_genes
}

In [ ]:
# some plots

In [ ]:
sc.pl.matrixplot(adata, [target_genes[0]], groupby=groupby, swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [target_genes[0]], groupby=groupby, swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(adata, [target_genes[1]], groupby=groupby, swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(adata, [target_genes[1]], groupby=groupby, swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

# Session Info

In [ ]:
sc.logging.print_versions()